In [1]:
import os
import time
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from datetime import datetime, timezone, timedelta
import numpy as np
import random
import logging
import argparse
import logging
import os
import numpy as np
import pandas as pd

In [2]:
# 시드 고정
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Set device
os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 경로 설정
ROOT_PATH = '/USER/kaggle/summarization'
DATA_DIR = '/USER/kaggle/summarization/data'
MODEL_DIR = ROOT_PATH

In [3]:
torch.cuda.is_available()
DEVICE

device(type='cuda')

In [4]:
# hyper-parameters
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16

# 학습 데이터만 있으니 학습 데이터셋 비율과 validation 데이터셋 비율을 나눔
TRAIN_RATIO = 0.9

In [6]:
import pandas as pd
from torch.utils.data import Dataset
from pytorch_pretrained_bert import BertTokenizer
from transformers import AutoTokenizer
from itertools import chain
import json
from transformers import ElectraModel, ElectraTokenizer

class CustomDataset(Dataset):
    def __init__(self, data_dir, mode):
        self.data_dir = data_dir
        self.mode = mode
        self.tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
        # self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        self.inputs, self.labels = self.data_loader()

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        
        if os.path.isfile(os.path.join(self.data_dir, self.mode + '_X.pt')):
            inputs = torch.load(os.path.join(self.data_dir, self.mode + '_X.pt'))
            labels = torch.load(os.path.join(self.data_dir, self.mode + '_Y.pt'))

        else:
            file_path = os.path.join(self.data_dir, 'train.json')
            df = pd.read_json(file_path, orient='records', encoding='utf-8-sig')
          
            if self.mode == 'train':
                df = df.loc[:TRAIN_RATIO*int(len(df)), :]
            elif self.mode == 'val':
                df = df.loc[TRAIN_RATIO*int(len(df)):, :]

            inputs = pd.DataFrame(columns=['src'])
            labels = pd.DataFrame(columns=['trg'])
            inputs['src'] =  df['article_original']
            labels['trg'] =  df['extractive']
          
            # Preprocessing
            inputs, labels = self.preprocessing(inputs, labels)
            # Save data
            torch.save(inputs ,os.path.join(self.data_dir, self.mode + '_X.pt'))
            torch.save(labels, os.path.join(self.data_dir, self.mode + '_Y.pt'))

        inputs = inputs.values
        labels = labels.values

        return inputs, labels

    def pad(self, data, pad_id, max_len):
        padded_data = data.map(lambda x : torch.cat([x, torch.tensor([pad_id] * (max_len - len(x)))]))
        return padded_data

    def preprocessing(self, inputs, labels):
        print('Preprocessing ' + self.mode + ' dataset..')
        #Encoding original text
        inputs['src'] = inputs['src'].map(lambda x: torch.tensor(list(chain.from_iterable([self.tokenizer.encode(x[i], max_length = int(512 / len(x)),  add_special_tokens=True) for i in range(len(x))]))))
        inputs['clss'] = inputs.src.map(lambda x : torch.cat([torch.where(x == 2)[0], torch.tensor([len(x)])]))
        inputs['segs'] = inputs.clss.map(lambda x : torch.tensor(list(chain.from_iterable([[0] * (x[i+1] - x[i]) if i % 2 == 0 else [1] * (x[i+1] - x[i]) for i, val in enumerate(x[:-1])]))))
        inputs['clss'] = inputs.clss.map(lambda x : x[:-1])

        ##Padding
        max_encoding_len = max(inputs.src.map(lambda x: len(x)))
        max_label_len = max(inputs.clss.map(lambda x: len(x)))
        inputs['src'] = self.pad(inputs.src, 0, max_encoding_len)
        inputs['segs'] = self.pad(inputs.segs, 0, max_encoding_len)
        inputs['clss'] = self.pad(inputs.clss, -1, max_label_len)
        inputs['mask'] = inputs.src.map(lambda x: ~ (x == 0))
        inputs['mask_clss'] = inputs.clss.map(lambda x: ~ (x == -1))

        #Binarize label {Extracted sentence : 1, Not Extracted sentence : 0}
        labels = labels['trg'].map(lambda  x: torch.tensor([1 if i in x else 0 for i in range(max_label_len)]))

        return inputs, labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return [self.inputs[index][i] for i in range(5)], self.labels[index]

In [10]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train')
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val')

train_dataloader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=EVAL_BATCH_SIZE, shuffle=False)

Loading train dataset..


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Preprocessing train dataset..
Loading val dataset..


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Preprocessing val dataset..


In [7]:
print(train_dataset.inputs[0])

[tensor([    2,    63,    21,    65,  3238,  4297,  7166,  4291,  4031,  4472,
         3288, 23695,  4084,  3288,  4057,  4255,    16,     3,     2,  3288,
        12191,  4084,  3288,  4057,  4255,  4112, 13021,  4110,  6984,  4418,
         4073,  4034, 13021,  4005,  4139,  4292,  8861,  4279,  4034, 12568,
         4292,  7453, 26509,  4474, 14227,    16,     3,     2, 13021, 10425,
         4070, 13021,  4110,  8637,  4283,  2468,  4073,  4034, 13021,  4005,
         4139,  4292,  8861,  4279,  4034, 12568,     3,     2,  6587, 15936,
         2799,  4372,  4073, 13021,  4467,  4029,    16,     3,     2, 15936,
         2967,  4258,    12,   583,  1892,    13,  3141, 33888,    16,     3,
            2,  2967,  4258,  4467,  4029,    12,   583,  1892,  1355,  1637,
           13,    16,     3,     2,  7935,  4467,  4029,  2728, 13021,  4106,
           12,  1369,  1683,  1277,    13,  3238,  2513,  4292,  9896,  4279,
         4219,    16,     3,     2, 13021,  4104,  4234,  3242,

In [8]:
# parameters
EPOCHS = 10
LEARNING_RATE = 5e-5
WEIGHT_DECAY = 0.00001
NUM_WORKERS = 2
EARLY_STOPPING_PATIENCE = 20

In [9]:
from torch import nn
import transformers
from sklearn.metrics import f1_score

class Summarizer(nn.Module):

    def __init__(self):
        """
        """
        super(Summarizer, self).__init__()
        self.encoder =ElectraModel.from_pretrained("monologg/koelectra-small-v3-discriminator")
        self.fc = nn.Linear(256, 1)
        # self.fc2 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x, segs, clss, mask, mask_clss, sentence_range=None):
        """
        """
        top_vec = self.encoder(input_ids = x.long(), attention_mask = mask.float(),  token_type_ids = segs.long()).last_hidden_state
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss.long()]
        sents_vec = sents_vec * mask_clss[:, :, None].float()
        # print(sents_vec.shape)
        h = self.fc(sents_vec).squeeze(-1)
        # sents_vec = self.sigmoid(h)
        # h = self.fc2(sents_vec).squeeze(-1)
        sent_scores = self.sigmoid(h) * mask_clss.float()
        return sent_scores
    

In [10]:
model = Summarizer().to(DEVICE)

In [11]:
def Hitrate(y_true, y_pred):
    """ Metric 함수 반환하는 함수

    Returns:
        metric_fn (Callable)
    """
    hitrate = np.array([len(list(set(ans).intersection(y_true[i])))/3 for i, ans in enumerate(y_pred)])
    score = np.mean(hitrate)
    return score

In [12]:
# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = torch.nn.BCELoss(reduction='none')

# Set metrics
metric_fn = Hitrate

In [13]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int, verbose: bool, logger:logging.RootLogger=None)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience
        self.verbose = verbose

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.logger = logger
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단

        Args:
            loss (float):

        Examples:
            
        Note:
            
        """  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopper, Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True

            if self.verbose:
                self.logger.info(msg) if self.logger else print(msg)
                
        elif loss <= self.min_loss:
            self.save_model = True
            msg = f"Early stopper, Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

            if self.verbose:
                self.logger.info(msg) if self.logger else print(msg)


In [14]:
class Trainer():
    """ Trainer
        epoch에 대한 학습 및 검증 절차 정의
    
    Attributes:
        model (`model`)
        device (str)
        loss_fn (Callable)
        metric_fn (Callable)
        optimizer (`optimizer`)
        scheduler (`scheduler`)
    """

    def __init__(self, model, device, loss_fn, metric_fn, optimizer=None, scheduler=None, logger=None):
        """ 초기화
        """
        self.model = model
        self.device = device
        self.loss_fn = loss_fn
        self.metric_fn = metric_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.logger = logger

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.train()
        self.train_total_loss = 0
        pred_lst = []
        target_lst = []
        for batch_index, (data, target) in enumerate(tqdm(dataloader)):
            self.optimizer.zero_grad()
            src = data[0].to(self.device)
            clss = data[1].to(self.device)
            segs = data[2].to(self.device)
            mask = data[3].to(self.device)
            mask_clss = data[4].to(self.device)
            target = target.float().to(self.device)
            sent_score = self.model(src, segs, clss, mask, mask_clss)
            loss = self.loss_fn(sent_score, target)
            loss = (loss * mask_clss.float()).sum()
            self.train_total_loss += loss
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
            try:
                target_lst.extend(torch.where(target==1)[1].reshape(-1,3).tolist())
            except:
                print(target)
                sys.exit()
                
        self.train_mean_loss = self.train_total_loss / len(dataloader)
        self.train_score = self.metric_fn(y_true=target_lst, y_pred=pred_lst)
        msg = f'Epoch {epoch_index}, Train, loss: {self.train_mean_loss}, Score: {self.train_score}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.eval()
        self.val_total_loss = 0
        pred_lst = []
        target_lst = []

        with torch.no_grad():
            for batch_index, (data, target) in enumerate(dataloader):
                src = data[0].to(self.device)
                clss = data[1].to(self.device)
                segs = data[2].to(self.device)
                mask = data[3].to(self.device)
                mask_clss = data[4].to(self.device)
                target = target.float().to(self.device)
                sent_score = self.model(src, segs, clss, mask, mask_clss)
                loss = self.loss_fn(sent_score, target)
                loss = (loss * mask_clss.float()).sum()
                self.val_total_loss += loss
                pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
                target_lst.extend(torch.where(target==1)[1].reshape(-1,3).tolist())
            self.val_mean_loss = self.val_total_loss / len(dataloader)
            self.validation_score = self.metric_fn(y_true=target_lst, y_pred=pred_lst)
            msg = f'Epoch {epoch_index}, Validation, loss: {self.val_mean_loss}, Score: {self.validation_score}'
            print(msg)

In [15]:
# Set trainer
trainer = Trainer(model, DEVICE, loss_fn, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE, verbose=True)

In [16]:
# TRAIN
from tqdm.auto import tqdm

start = time.time()
criterion = 0

for epoch_index in tqdm(range(EPOCHS)):
    
    trainer.train_epoch(train_dataloader, epoch_index=epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index=epoch_index)
   
    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if trainer.validation_score > criterion:
        criterion = trainer.validation_score
        check_point = {
            'model' : model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        
        torch.save({
            'epoch': epoch_index,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': trainer.val_mean_loss,
            }, os.path.join(MODEL_DIR, 'best2.pt'))
        
        
print("train finished, best2.pt saved.")


Epoch 0, Train, loss: 86.32435607910156, Score: 0.43491329479768787
Epoch 0, Validation, loss: 80.58061218261719, Score: 0.46558423535942267



Epoch 1, Train, loss: 82.4341812133789, Score: 0.4351290944123314
Epoch 1, Validation, loss: 78.86117553710938, Score: 0.5267832361920622
Early stopper, Validation loss decreased 80.58061218261719 -> 78.86117553710938



Epoch 2, Train, loss: 77.6243896484375, Score: 0.5444315992292871
Epoch 2, Validation, loss: 69.96427917480469, Score: 0.5936719400499584
Early stopper, Validation loss decreased 78.86117553710938 -> 69.96427917480469



Epoch 3, Train, loss: 71.06617736816406, Score: 0.6024354527938341
Epoch 3, Validation, loss: 67.65601348876953, Score: 0.6297529836247571
Early stopper, Validation loss decreased 69.96427917480469 -> 67.65601348876953


KeyboardInterrupt: 

In [8]:
submit= pd.read_csv(os.path.join(DATA_DIR,'sample_submission.csv'))
submit.head()

,ID,idx_0,idx_1,idx_2,idx_3,idx_4,idx_5,idx_6,idx_7,idx_8,...,idx_39,idx_40,idx_41,idx_42,idx_43,idx_44,idx_45,idx_46,idx_47,idx_48
0,79095,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,204506,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,142079,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,110816,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,207249,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# 테스트 데이터셋 클래스 정의

class TestDataset(Dataset):
    """ CustomDataset과 비슷한 구조이지만 레이블이 주어지지 않음을 염두 """

    def __init__(self, data_dir, mode):
        self.data_dir = data_dir
        self.mode = mode
        # self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        self.tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
        self.inputs = self.data_loader()

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if os.path.isfile(os.path.join(self.data_dir, self.mode+'_X.pt')):
            # torch tensor 불러오기
            inputs = torch.load(os.path.join(self.data_dir, self.mode + '_X.pt'))
        else:
            # json 파일 불러오기
            file_path = os.path.join(self.data_dir, self.mode + '.json')
            df = pd.read_json(file_path, orient='records', encoding='utf-8-sig')
            inputs = pd.DataFrame(columns=['src'])
            inputs['src'] =  df['article_original']
      
            # 전처리
            inputs = self.preprocessing(inputs)
            
            # 다음부터는 전처리 과정을 반복하지 않기 위해 tensor 저장
            torch.save(inputs ,os.path.join(self.data_dir, self.mode + '_X.pt'))

        inputs = inputs.values

        return inputs

    def pad(self, data, pad_id, max_len):
        padded_data = data.map(lambda x : torch.cat([x, torch.tensor([pad_id] * (max_len - len(x)))]))
        return padded_data

    def preprocessing(self, inputs):
        print('Preprocessing ' + self.mode + ' dataset..')
        
        #Encoding original text
        inputs['src'] = inputs['src'].map(lambda x: torch.tensor(list(chain.from_iterable([self.tokenizer.encode(x[i], max_length = int(512 / len(x)),  add_special_tokens=True) for i in range(len(x))]))))
        inputs['clss'] = inputs.src.map(lambda x : torch.cat([torch.where(x == 2)[0], torch.tensor([len(x)])]))
        inputs['segs'] = inputs.clss.map(lambda x : torch.tensor(list(chain.from_iterable([[0] * (x[i+1] - x[i]) if i % 2 == 0 else [1] * (x[i+1] - x[i]) for i, val in enumerate(x[:-1])]))))
        inputs['clss'] = inputs.clss.map(lambda x : x[:-1])

        ##Padding
        max_encoding_len = max(inputs.src.map(lambda x: len(x)))
        max_label_len = max(inputs.clss.map(lambda x: len(x)))
        inputs['src'] = self.pad(inputs.src, 0, max_encoding_len)
        inputs['segs'] = self.pad(inputs.segs, 0, max_encoding_len)
        inputs['clss'] = self.pad(inputs.clss, -1, max_label_len)
        inputs['mask'] = inputs.src.map(lambda x: ~ (x == 0))
        inputs['mask_clss'] = inputs.clss.map(lambda x: ~ (x == -1))
     
        return inputs

    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return [self.inputs[index][i] for i in range(5)]


In [9]:
# 테스트 데이터 로드
test_dataset = TestDataset(data_dir=DATA_DIR, mode = 'test')
test_dataloader = DataLoader(dataset=test_dataset, batch_size=EVAL_BATCH_SIZE, shuffle=False)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Loading test dataset..
Preprocessing test dataset..


In [ ]:
DEVICE

In [ ]:
""" 이전에 학습한 모델 weight파일을 불러 추론하려면 아래 주석을 풀고 실행
    학습 진행 후 바로 추론하는 경우 학습 과정의 model 사용 (주석 풀지 않고 실행) """
MODEL_DIR = os.path.join(ROOT_PATH, 'best2.pt')
model = Summarizer().to(DEVICE)
model.load_state_dict(torch.load(MODEL_DIR)['model_state_dict'])

# 추론
model.eval()

# 추론 결과를 pred 리스트로 저장할 예정
pred_lst = []

with torch.no_grad():
    for batch_index, (data) in enumerate(test_dataloader):
        src = data[0].to(DEVICE)
        clss = data[1].to(DEVICE)
        segs = data[2].to(DEVICE)
        mask = data[3].to(DEVICE)
        mask_clss = data[4].to(DEVICE)
        sent_score = model(src, segs, clss, mask, mask_clss)
        pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
            
        # 진행과정 출력
        if batch_index % 150 == 0:
            print(f'Prediction: {batch_index}/{len(test_dataloader)} completed')
    print("Prediction all completed")


In [ ]:
print(pred_lst[:5])

In [ ]:
# 제출 파일
for i,txt in enumerate(pred_lst):
    submit.iloc[i,txt[0]+1] += 1
    submit.iloc[i,txt[1]+1] += 1
    submit.iloc[i,txt[2]+1] += 1
submit.head()

In [10]:
submit.to_csv(os.path.join(ROOT_PATH, 'prediction4.csv'), index=False)